# Breast cancer detection from thermal imaging

The main purpose of this project is to develop a comprehensive decision support system for breast cancer screening.

## Library import
En esta sección se tratará de importar las librerias que se van a utilizar a lo largo de este modelo. Hay que tener en cuenta que parte de las librerías que utiliza este programa se encuentran declaradas en los ficheros que se encuentran en `src/scripts/*.py`.

In [1]:
from scripts import *

In [2]:
computer.check_available_devices() # Check available devices

Available devices:
- CPU: /physical_device:CPU:0
- GPU: /physical_device:GPU:0


## Data selection
To make this model work correctly it will be necessary to extract and save the images found in the `data` folder.

In this folder there are two labeled folders that contain all the images to be used:
```
├── healthy
└── sick
```

In [3]:
data = Data("./data/")